## Neo4j Lanchain을 활용한 영화 추천 시스템

### 1. Neo4j AuraDB 환경설정

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_neo4j import Neo4jGraph

#LangChain 도구 활용 - DB연결 객체 초기화
graph = Neo4jGraph(
    url = os.getenv("NEO4J_URI"),
    username = os.getenv("NEO4J_USERNAME"),
    password = os.getenv("NEO4J_PASSWORD"),
)

In [4]:
def reset_database(graph):
    """데이터베이스 초기화하기"""
    #모든 노드와 관계 삭제
    graph.query("MATCH (n) DETACH DELETE n")

    #모든 제약조건 삭제
    constraints = graph.query("SHOW CONSTRAINTS")
    for constraint in constraints:
        constraint_name = constraint.get("name")
        if constraint_name :
            graph.query(f"DROP CONSTRAINT {constraint_name}")
    
    #모든 인덱스 삭제
    indexes = graph.query("SHOW INDEXES")
    for index in indexes:
        index_name = index.get("name")
        index_type = index.get("type")
        if index_name and index_type != "CONSTRAINT":
            graph.query(f"DROP INDEX {index_name}")
    
    print("데이터베이스가 초기화 되었습니다.")


reset_database(graph)


데이터베이스가 초기화 되었습니다.


## 2. 지식 그래프 모델링 및 구축

### 2.1 온톨로지(Ontology) 정의 및 스키마 설계

* 온톨로지는 특정 도메인 내 개념과 그 관계를 체계적으로 표현하는 지식구조이다. Neo4j에서는 제약조건과 인덱스를 활용하여 데이터 모델을 구현할 수 있다.

In [5]:
# 제약조건 생성 (데이터 무결성 보장)
# 각 노드 타입별로 고유성 제약조건을 설정하여 중복 데이터 방지

constraints = [
    "CREATE CONSTRAINT movie_id_unique IF NOT EXISTS FOR (m:Movie) REQUIRE m.id IS UNIQUE",
    "CREATE CONSTRAINT person_name_unique IF NOT EXISTS FOR (p:Person) REQUIRE p.name IS UNIQUE",
    "CREATE CONSTRAINT genre_namne_unique IF NOT EXISTS FOR (g:Genre) REQUIRE g.name IS UNIQUE"
]

# 인덱스 생성 (검색 성능 최적화)
# 자주 검색되는 속성에 인덱스를 생성하여 쿼리 성능 향상
indexes = [
    "CREATE INDEX movie_title_index IF NOT EXISTS FOR (m:Movie) ON (m.title)",
    "CREATE INDEX movie_release_index IF NOT EXISTS FOR (m:Movie) ON (m.released)",
]

# 제약조건 및 인덱스 실행
# 정의된 모든 제약조건을 Neo4j 데이터베이스에 적용
for constraint in constraints :
    graph.query(constraint)


# 정의된 모든 인덱스를 Neo4j 데이터베이스에 적용
for index in indexes:
    graph.query(index)

print("스키마 설정 완료")

스키마 설정 완료


### 2.2 CSV 데이터로 지식 그래프 구축

In [10]:
# csv 파일 읽기
import pandas as pd
df = pd.read_csv('data/movies_tmdb_small.csv')
print(df.shape)

df.head(2)

TypeError: Cannot convert numpy.ndarray to numpy.ndarray